# Model transforming words into a number
- input: sequence of letters
- output: number 


Examples: 
- input: sequence 'one hundred' output: 100
- input: sequence 'two hundred fourteen' output: 214

The code in file number2words.py taken from: https://www.codesansar.com/python-programming-examples/number-words-conversion-no-library-used.htm

In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
import numpy as np
from number2words import getWords

## Model

In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(None,1),return_sequences=True)) # sequences of singlen numbers
model.add(LSTM(128))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer="adam",metrics=['mae','mse'])
num_epochs = 0
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 128)         66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 198,273
Trainable params: 198,273
Non-trainable params: 0
_________________________________________________________________


## Dataset creation

### Helper methods

In [3]:
# helper method, converts sequence of numbers to text
def to_text(sample):
    return ''.join([idx2char[int(x)] for x in sample])
# helper method, converts text to sequence of numbers
def to_number(words):
    return np.array([char2idx[char] for char in words])

### Dataset - **samples** and **labels**

In [4]:
DATASET_SIZE=200

samples = []
labels = []
all_words = ''
max_len = 0
for i in range(DATASET_SIZE):
    labels.append(i)
    #words = lslownie(i)
    words = getWords(i)
    samples.append(words)
    all_words += words
    if len(words)>max_len: 
        max_len = len(words)
  
print('Max len of text',max_len)
vocab = sorted(set(all_words))
vocab_size = len(vocab)
print('vocabulary (used letters)',vocab)
print ('unique characters',vocab_size)

Max len of text 27
vocabulary (used letters) [' ', 'd', 'e', 'f', 'g', 'h', 'i', 'l', 'n', 'o', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
unique characters 19


#### Creating a mapping from unique characters to indices

In [5]:
char2idx = {char:index for index, char in enumerate(vocab)}
print('char2idx:\n',char2idx)
idx2char = np.array(vocab)
print('idx2char\n',idx2char)

char2idx:
 {' ': 0, 'd': 1, 'e': 2, 'f': 3, 'g': 4, 'h': 5, 'i': 6, 'l': 7, 'n': 8, 'o': 9, 'r': 10, 's': 11, 't': 12, 'u': 13, 'v': 14, 'w': 15, 'x': 16, 'y': 17, 'z': 18}
idx2char
 [' ' 'd' 'e' 'f' 'g' 'h' 'i' 'l' 'n' 'o' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y'
 'z']


#### Convert letters to numbers using char2idx

In [6]:
samples_int = []
for s in samples:
    v = np.array([char2idx[char] for char in s])
    samples_int.append(v) # different sizes!
print(samples[123],' ->becomes-> ',samples_int[123])

one hundred twenty three    ->becomes->  [ 9  8  2  0  5 13  8  1 10  2  1  0 12 15  2  8 12 17  0 12  5 10  2  2
  0  0]


#### From list of lists to numpy - must have a fixed number of characters (30 -> max_len)

In [7]:
samples = np.zeros((DATASET_SIZE,max_len))
for i in range(len(samples_int)):
    for j in range(len(samples_int[i])):
        samples[i,j] = np.array(samples_int[i][j]) # all not used have '0' which is ' '
print('SAMPLES\n\n',samples)
print(samples.shape)

SAMPLES

 [[18.  2. 10. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [12. 15.  9. ...  0.  0.  0.]
 ...
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]
 [ 9.  8.  2. ...  0.  0.  0.]]
(200, 27)


In [8]:
samples = np.expand_dims(samples,axis=2) #add the third dimension
labels = np.array(labels,dtype=float)

print("Sample (for 123):\n",samples[123])
print("Sample decoded",to_text(samples[123]))
print("Label (output):",labels[123])

print('samples shape',samples.shape)
print('labels shape',labels.shape)

Sample (for 123):
 [[ 9.]
 [ 8.]
 [ 2.]
 [ 0.]
 [ 5.]
 [13.]
 [ 8.]
 [ 1.]
 [10.]
 [ 2.]
 [ 1.]
 [ 0.]
 [12.]
 [15.]
 [ 2.]
 [ 8.]
 [12.]
 [17.]
 [ 0.]
 [12.]
 [ 5.]
 [10.]
 [ 2.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]]
Sample decoded one hundred twenty three   
Label (output): 123.0
samples shape (200, 27, 1)
labels shape (200,)


In [9]:
TRAINING_SIZE = .5
from sklearn.model_selection import train_test_split
(trainSamples, testSamples, trainLabels, testLabels) = train_test_split(samples, labels,train_size=TRAINING_SIZE, random_state=1)
print('Training samples:',len(trainSamples),' test samples',len(testSamples))

Training samples: 100  test samples 100


In [11]:
EPOCHS=100
BATCH_SIZE = int(len(trainSamples)/4)
print('Training with',len(trainSamples),'samples',EPOCHS,'epochs and batch_size=',BATCH_SIZE)
for x in range(10):
    H = model.fit(trainSamples, trainLabels, epochs=EPOCHS,verbose=0,batch_size=BATCH_SIZE)
    num_epochs += EPOCHS
    print("\n{}/10 Epochs: {} - loss={:6.3f}, loss improvement={:6.3f}".
          format(x, num_epochs,H.history['loss'][-1], H.history['loss'][0]-H.history['loss'][-1]))
    check_model()
print("Done")

Training with 100 samples 100 epochs and batch_size= 25

0/10 Epochs: 100 - loss=4690.518, loss improvement=7341.845
text => [predicted value] error=[error]
ninety five                 =>  55.83 error = 39.17
ninety eight                =>  55.83 error = 42.17
eighty                      =>  55.83 error = 24.17
eighty eight                =>  55.83 error = 32.17
one hundred fifteen         =>  55.83 error = 59.17
one hundred thirteen        =>  55.83 error = 57.17
ninety nine                 =>  55.83 error = 43.17
one hundred eighteen        =>  55.83 error = 62.17
one hundred ninety one      =>  55.83 error = 135.17
one hundred nineteen        =>  55.83 error = 63.17
Mean error = 59.53542

1/10 Epochs: 200 - loss=3306.549, loss improvement=1358.142
text => [predicted value] error=[error]
twenty six                  =>  81.23 error = 55.23
one hundred fourteen        =>  81.23 error = 32.77
one hundred sixty four      =>  81.23 error = 82.77
seventy four                =>  81.23 error

In [12]:
import random

def check_model(verbose=0,how_many=10):
    pred = model.predict(samples)
    print('text => [predicted value] error=[error]')
    error = []
    for i in range(len(pred)):
        res = samples[i]
        error.append(abs(i-pred[i]))
        if verbose==1:
            train = ''
            if i in trainLabels: train='[T]'
            print(i,to_text(res),'=> {:.2f} error = {:.2f}'.format(pred[i,0],abs(i-pred[i,0])),train)
    if verbose<1: # if not verbose just display 'how_many' random samples
        for i in range(how_many):        
            x = random.randrange(DATASET_SIZE)
            res = samples[x]
            print(to_text(res),'=>  {:.2f} error = {:.2f}'.format(pred[x,0],abs(x-pred[x,0])))      
    print('Mean error =',np.mean(error))        
    return np.mean(error)
check_model(1)

text => [predicted value] error=[error]
0 zero                        => 4.34 error = 4.34 [T]
1 one                         => 6.63 error = 5.63 [T]
2 two                         => 4.27 error = 2.27 [T]
3 three                       => 5.80 error = 2.80 [T]
4 four                        => 15.63 error = 11.63 
5 five                        => 6.96 error = 1.96 
6 six                         => 5.29 error = 0.71 [T]
7 seven                       => 6.39 error = 0.61 [T]
8 eight                       => 7.97 error = 0.03 [T]
9 nine                        => 6.75 error = 2.25 [T]
10 ten                         => 6.57 error = 3.43 [T]
11 eleven                      => 8.13 error = 2.87 
12 twelve                      => 3.92 error = 8.08 
13 thirteen                    => 15.38 error = 2.38 
14 fourteen                    => 32.13 error = 18.13 
15 fifteen                     => 14.72 error = 0.28 [T]
16 sixteen                     => 29.80 error = 13.80 
17 seventeen                   

7.4975524

In [19]:

x = to_number('one hundred thirty nine      ')
#x = to_number('one hundred ninety nine     ')

#              xxxxxxxxxxxxxxxxxxxxxxxxxxxxx
x = np.expand_dims(x,axis=1)
x = np.expand_dims(x,axis=0)
model.predict(x)

array([[116.47714]], dtype=float32)

In [ ]:
model.save('model_words2numbers.h5')